In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from conf.conf_analysis import protca
from utils.atomselect import interval_selection
from plot.plot_utilities import hist1d, edgeformat, savefig

In [ ]:
%store -r traj_ids
%store -r traj_ids_closed

%store -r map_assign
%store -r color_assign

# Helix segmental distance

In [ ]:
def helix_segment_distance(segment1, segment2, traj_ids):
    segment1_resids = interval_selection(*segment1)
    segment2_resids = interval_selection(*segment2)

    main = protca(traj_ids, np.concatenate([segment1_resids, segment2_resids]))
    main.load_cainfo()

    segment1_xyz = main.ca_coord_set.query('resid in @segment1_resids')[['x', 'y', 'z']].values
    segment1_xyz = segment1_xyz.reshape(-1, len(segment1_resids), 3)
    com_segment1 = np.mean(segment1_xyz, axis=1)

    segment2_xyz = main.ca_coord_set.query('resid in @segment2_resids')[['x', 'y', 'z']].values
    segment2_xyz = segment2_xyz.reshape(-1, len(segment2_resids), 3)
    com_segment2 = np.mean(segment2_xyz, axis=1)

    segment_distance = np.linalg.norm(com_segment1 - com_segment2, axis=1)
    return segment_distance

In [ ]:
tm10top = [1013, 1021]
tm11top = [1114, 1122]

tm10_tm11_topdist_nearopen = helix_segment_distance(tm10top, tm11top, traj_ids)
tm10_tm11_topdist_closed = helix_segment_distance(tm10top, tm11top, traj_ids_closed)

# TM10-TM11

In [ ]:
%store -r states_df

fig, axs = plt.subplots()
edgeformat(axs)

for s in [3,0,2,1]:
    states_selection = (states_df['tmpc1v2'] == s)
    hist1d(tm10_tm11_topdist_nearopen[states_selection], bins=50).plot(axs, label=map_assign[s], color=color_assign[s])

hist1d(tm10_tm11_topdist_closed, bins=50).plot(axs, label="md_5UAK", color='black')

axs.set_xlim(8,20)
axs.set_ylim(0,1)
axs.set_xlabel(r"$d_{10-11}$ [$\mathrm{\AA}$]", fontsize=16)
axs.set_ylabel("Prob. density [A.U.]", fontsize=16)
axs.legend()
axs.grid(True, ls='--')

# savefig("dist_tm10-tm11_top.pdf")